In [1]:
%cd /Users/aravindhswaminathan/Documents/GitHub/TwoSidedYSYK
#!pwd
import numpy as np
from matplotlib import pyplot as plt
from SYK_fft import *
import sys
from scipy import optimize

/Users/aravindhswaminathan/Documents/GitHub/TwoSidedYSYK


In [2]:
Nbig = int(2**18)
err = 1e-5

beta = 5000.
mu = 0.001
J = 1.

omega = (2 * np.arange(Nbig) - Nbig + 1) * np.pi/beta
nu = (2 * np.arange(Nbig) - Nbig ) * np.pi/beta
tau = (np.arange(Nbig) + 1./2) * beta/Nbig

In [3]:
#Gtau = -0.5*np.ones(Nbig) 
#Gtau = 1/(4*np.pi*J**2) * (np.pi/(beta*np.sin(np.pi*tau/beta)))**0.5
#init_Gomega = Time2FreqF(Gtau,Nbig,beta)
#init_Gomega = Time2FreqF(Gtau,Nbig,beta)
init_Gomega = 1./(1j*omega + mu)
#Gtau = Freq2TimeF(1/(1j*omega + mu),Nbig,beta)
''' It's crucial to start with this, even changing the sign here is enough to make the solution 
fall into an unwanted one'''

itern = 0
diff = 1.
x = 0.2

def fun(Gomega):
    Gtau = Freq2TimeF(Gomega,Nbig,beta)
    Sigmatau = J**2 * Gtau**2 * Gtau[::-1]
    Sigmaomega = Time2FreqF(Sigmatau,Nbig,beta)
    return(Gomega - 1./(1j*omega + mu - Sigmaomega))
    

sol = optimize.anderson(fun, init_Gomega, M=5, verbose = False, f_tol = 1e-2, maxiter=500)
Gomega = sol 
Gtau = Freq2TimeF(Gomega,Nbig,beta)

# while(diff>err):
#     itern+=1
#     diffold = diff
    
#     oldGtau = 1.0*Gtau
#     oldGomega = Time2FreqF(oldGtau,Nbig,beta)
    
#     Sigmatau = J**2 * Gtau**2 * Gtau[::-1] #KMS G(-tau) = -G(beta-tau)
#     Sigmaomega = Time2FreqF(Sigmatau,Nbig,beta)
    
#     Gomega = x*(1./(1j*omega + mu - Sigmaomega)) + (1-x)*oldGomega
#     #Gtau = Freq2TimeF(Gomega - (1/(1j*omega)),Nbig,beta) - 0.5 #cool trick for Gibbs oscillations
#     Gtau = Freq2TimeF(Gomega,Nbig,beta)
    
#     diff = np. sqrt((1.0/Nbig) * np.sum((np.abs(Gtau-oldGtau))**2)) 
#     if diff>diffold:
#          x/=2


Q = 0.5*(Gtau[0] - Gtau[-1])
#print("itern = ",itern," , diff = ", diff, " , x = ", x)
print("Q = ", np.real(Q))

KeyboardInterrupt: 

In [ ]:
plt.plot(tau/beta, np.real(Gtau), 'r', label = 'numerics Gtau')
#plt.plot(tau/beta, np.real(Gconftau), 'b--', label = 'analytical Gtau' )
plt.ylim(-1,1)
plt.xlabel(r'$\tau/\beta$',labelpad = 0)
plt.legend()
plt.title('Q = %.3f ' %np.real(Q) )

In [ ]:
plt.plot(omega,np.imag(Gomega),'p')
plt.plot(omega,np.zeros_like(omega), color = 'gray', linestyle = '--')
plt.axvline(0,color='gray',linestyle = '--')
plt.xlim(-5,5)
plt.xlabel(r'$\omega_n$')
plt.ylabel(r'$G(i\omega_n)$')
plt.title(r'$\mu = $ ' + str(mu) + ' ;  Q = %.3f' %np.real(Q))

In [ ]:
plt.loglog(omega[Nbig//2:],-np.imag(Gomega[Nbig//2:]),'.')
fudfac = -np.imag(Gomega[Nbig//2])/omega[Nbig//2]**(-0.5)
plt.loglog(omega[Nbig//2:],fudfac* (omega[Nbig//2:]**(-0.5)),'k--')
plt.title('Checking power law at low frequencies')
plt.xlabel(r'$\omega_n$')
plt.ylabel(r'$-\Im{G(i\omega_n)}$')
plt.xlim(5e-4,5e0)
plt.ylim(5e-2,5e2)
plt.gca().set_aspect('equal')
plt.figure(figsize=(6,6))

## Checking the anderson algorithm for a simple case of a function of a single variable

In [ ]:
sol = optimize.anderson(lambda x : x**2 - 9, [2,], verbose=True)
sol